In [7]:
import pandas as pd 
import numpy as np 
import scipy
import xlrd 
import sklearn

In [3]:
clinc_table1 = pd.read_csv('../data/processed/process_clin_1.csv')
clinc_table2 = pd.read_csv('../data/processed/process_clin_2.csv')
clinc_table3 = pd.read_csv('../data/processed/process_clin_1_and_2.csv')

# gene_table = pd.read_csv('../data/processed/process_gene_pathway_can100_66.csv')
# gene_table = pd.read_csv('../data/processed/process_gene_pathway_dise100_66.csv')
gene_table1 = pd.read_csv('../data/processed/process_gene_single_P1.csv')
gene_table2 = pd.read_csv('../data/processed/process_gene_single_P2.csv')

In [5]:
# lip_table_specie = pd.read_csv('../data/processed/process_lip_group_binary_v1.csv')
# lip_table_specie = pd.read_csv('../data/processed/process_lip_group_binary_v2.csv')
# lip_table_specie = pd.read_csv('../data/processed/process_lip_group_v3.csv')
lip_table_specie_binary = pd.read_csv('../data/processed/process_lip_specie_binary_single_P1_P2_sig.csv')
lip_table_specie_binary_standard = pd.read_csv('../data/processed/process_lip_specie_standard_binary_single_P1_P2_sig.csv')
lip_table_specie_continues = pd.read_csv('../data/processed/process_lip_specie_cont_single_P1_P2_sig.csv')

# lip_table_total = pd.read_csv('../data/processed/process_lip_total_binary_single.csv')



In [6]:
# drop the duplicate term, only use the first one
gene_table1= gene_table1.drop_duplicates(subset=['Patient_ID'],keep='first')
print('# duplicate gene samples of P1 is',len(set(gene_table1['Patient_ID']))-len((gene_table1['Patient_ID'])))

gene_table2= gene_table2.drop_duplicates(subset=['Patient_ID'],keep='first')
print('# duplicate gene samples of P2 is',len(set(gene_table2['Patient_ID']))-len((gene_table2['Patient_ID'])))


# duplicate gene samples of P1 is 0
# duplicate gene samples of P2 is 0


In [26]:
# not use clic feafures

# samples at point1: normalize the clinic numerical features, except the time_gap, live/death label temporaly, we will add them at the end of merged all future
# clinc_table1_new = pd.DataFrame()

# clinc_table1_new['Patient_ID'] = clinc_table1['Patient_ID']
# num_feature_list = ['sdms psa at sample 1','sdms ldh at sample 1','BMI at time of collection']

# for name in num_feature_list:
#     df = clinc_table1[name]
#     clinc_table1_new['clin_' + name] = (df-df.min())/(df.max()-df.min())
# clinc_table1_new

# number of numerical features for samples at point3&4: 5

In [8]:
# targets for samples at point1:
clinc_target_1 = pd.DataFrame()

clinc_target_1['Patient_ID'] = clinc_table1['Patient_ID']
num_feature_list = ['ADT_start_fail','ADT_fail_relaspe','ADT_start_death','1= death; 0=alive','gap_death_consent','gap_surv_time_class','ADT_if_fail']

for name in num_feature_list:
    clinc_target_1[name] = clinc_table1[name]

clinc_target_1


,Patient_ID,ADT_start_fail,ADT_fail_relaspe,ADT_start_death,1= death; 0=alive,gap_death_consent,gap_surv_time_class,ADT_if_fail
0,1015,NaN,NaN,3039.0,0.0,3053,1,0
1,1027,NaN,NaN,959.0,1.0,959,0,0
2,1028,1734.0,54.0,3208.0,1.0,3219,1,2
3,1040,NaN,NaN,2323.0,0.0,2323,1,0
4,1048,897.0,67.0,2035.0,1.0,2073,2,2
...,...,...,...,...,...,...,...,...
94,1528,NaN,NaN,1472.0,1.0,1466,2,0
95,1530,NaN,NaN,34.0,0.0,1,0,0
96,1531,NaN,NaN,141.0,0.0,174,0,0
97,1532,NaN,NaN,1524.0,0.0,1694,2,0


In [9]:
# targets for samples at point3%4:
clinc_target_2 = pd.DataFrame()

clinc_target_2['Patient_ID'] = clinc_table2['Patient_ID']
num_feature_list = ['gap_death_consent','1= death; 0=alive','gap_surv_time_class']

for name in num_feature_list:
    clinc_target_2[name] = clinc_table2[name]

clinc_target_2

,Patient_ID,gap_death_consent,1= death; 0=alive,gap_surv_time_class
0,1001,2062,1.0,1
1,1002,524,1.0,0
2,1003,208,1.0,0
3,1004,2390,1.0,1
4,1005,286,1.0,0
...,...,...,...,...
245,1517,145,1.0,0
246,1519,752,1.0,2
247,1520,1341,1.0,2
248,1524,1694,0.0,1


In [10]:
lip_table_specie_continues.shape # num of lip_group feature: 308(binary), 614(continues)

(807, 308)

In [11]:
lip_table_specie_binary.shape # num of lip_group feature: 308(binary), 614(continues)

(807, 614)

In [13]:
# full-feature-data 2: point1_gene + lip_specie_binary_standard

merge_key = 'Patient_ID'
new1 = pd.merge(lip_table_specie_binary_standard,gene_table1,on = merge_key)
# new2 = pd.merge(new1, clinc_table1_new,on = merge_key)
p1_spicie_binary_standard = pd.merge(new1, clinc_target_1,on = merge_key)

print('overlapping patients:',-len(set(p1_spicie_binary_standard['Patient_ID']))+len((p1_spicie_binary_standard['Patient_ID'])))



# new3.to_csv('../data/processed/all_feature_p1_lip_specie_single.csv',index=False)
p1_spicie_binary_standard.to_csv('../data/processed/all_feature_p1_spicie_binary_standard_single.csv',index=False)

p1_spicie_binary_standard



overlapping patients: 0


,Patient_ID,lip_P2_Sph_Sph(d16:1),lip_P2_Sph_Sph(d18:1),lip_P2_Sph_Sph(d18:2),lip_P1_Cer_Cer(d16:1/16:0),lip_P2_Cer_Cer(d16:1/16:0),lip_P1_Cer_Cer(d16:1/18:0),lip_P2_Cer_Cer(d16:1/18:0),lip_P1_Cer_Cer(d16:1/20:0),lip_P2_Cer_Cer(d16:1/20:0),...,gene_P1_BRCA1_SNP,gene_P1_BRCA2_SNP,gene_P1_CHEK2_SNP,ADT_start_fail,ADT_fail_relaspe,ADT_start_death,1= death; 0=alive,gap_death_consent,gap_surv_time_class,ADT_if_fail
0,1028,0,0,0,0,0,0,0,0,0,...,0,0,0,1734.0,54.0,3208.0,1.0,3219,1,2
1,1066,0,0,0,1,1,0,0,0,0,...,0,0,0,NaN,NaN,NaN,0.0,2464,1,0
2,1040,0,1,0,0,0,1,1,1,1,...,0,1,0,NaN,NaN,2323.0,0.0,2323,1,0
3,1058,0,1,0,0,0,0,0,0,0,...,0,0,1,318.0,0.0,581.0,1.0,581,0,2
4,1059,0,0,0,0,0,0,0,0,0,...,0,0,0,NaN,NaN,2608.0,0.0,2608,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,1530,1,1,1,1,1,1,1,1,1,...,0,0,0,NaN,NaN,34.0,0.0,1,0,0
67,1531,1,1,1,0,0,0,0,0,0,...,0,0,0,NaN,NaN,141.0,0.0,174,0,0
68,1522,1,0,1,1,1,1,1,1,1,...,0,0,0,NaN,NaN,3750.0,1.0,706,0,0
69,1514,1,1,0,0,0,0,0,0,0,...,0,0,0,2355.0,NaN,3453.0,1.0,1098,0,2


In [17]:
# full-feature-data 2: point1_gene + lip_specie_binary

merge_key = 'Patient_ID'
new1 = pd.merge(lip_table_specie_binary,gene_table1,on = merge_key)
# new2 = pd.merge(new1, clinc_table1_new,on = merge_key)
p1_spicie_binary = pd.merge(new1, clinc_target_1,on = merge_key)

print('overlapping patients:',-len(set(p1_spicie_binary['Patient_ID']))+len((p1_spicie_binary['Patient_ID'])))



# new3.to_csv('../data/processed/all_feature_p1_lip_specie_single.csv',index=False)
p1_spicie_binary.to_csv('../data/processed/all_feature_p1_spicie_binary_single.csv',index=False)

p1_spicie_binary


overlapping patients: 0


,Patient_ID,lip_P2_Sph(d16:1)_Sph_high_25,lip_P2_Sph(d16:1)_Sph_low_25,lip_P2_Sph(d18:1)_Sph_high_25,lip_P2_Sph(d18:1)_Sph_low_25,lip_P2_Sph(d18:2)_Sph_high_25,lip_P2_Sph(d18:2)_Sph_low_25,lip_P1_Cer_Cer(d16:1/16:0)_high,lip_P1_Cer_Cer(d16:1/16:0)_low,lip_P2_Cer(d16:1/16:0)_Cer_high_5,...,gene_P1_BRCA1_SNP,gene_P1_BRCA2_SNP,gene_P1_CHEK2_SNP,ADT_start_fail,ADT_fail_relaspe,ADT_start_death,1= death; 0=alive,gap_death_consent,gap_surv_time_class,ADT_if_fail
0,1028,0,1,0,1,0,1,0,1,0,...,0,0,0,1734.0,54.0,3208.0,1.0,3219,1,2
1,1066,0,1,0,1,0,1,1,0,1,...,0,0,0,NaN,NaN,NaN,0.0,2464,1,0
2,1040,0,1,1,0,0,1,0,1,0,...,0,1,0,NaN,NaN,2323.0,0.0,2323,1,0
3,1058,0,1,1,0,0,1,0,1,0,...,0,0,1,318.0,0.0,581.0,1.0,581,0,2
4,1059,0,1,0,1,0,1,0,1,0,...,0,0,0,NaN,NaN,2608.0,0.0,2608,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,1530,1,0,1,0,1,0,1,0,1,...,0,0,0,NaN,NaN,34.0,0.0,1,0,0
67,1531,1,0,1,0,1,0,0,1,0,...,0,0,0,NaN,NaN,141.0,0.0,174,0,0
68,1522,1,0,0,1,1,0,1,0,1,...,0,0,0,NaN,NaN,3750.0,1.0,706,0,0
69,1514,1,0,1,0,0,1,0,1,0,...,0,0,0,2355.0,NaN,3453.0,1.0,1098,0,2


In [14]:
# full-feature-data : point1_gene + lip_specie_continues

merge_key = 'Patient_ID'
new1 = pd.merge(lip_table_specie_continues,gene_table1,on = merge_key)
# new2 = pd.merge(new1, clinc_table1_new,on = merge_key)
p1_spicie_cont_single = pd.merge(new1, clinc_target_1,on = merge_key)

print('overlapping patients:',-len(set(p1_spicie_cont_single['Patient_ID']))+len((p1_spicie_cont_single['Patient_ID'])))



# new3.to_csv('../data/processed/all_feature_p1_lip_specie_single.csv',index=False)
p1_spicie_cont_single.to_csv('../data/processed/all_feature_p1_spicie_cont_single.csv',index=False)

p1_spicie_cont_single



overlapping patients: 0


,Patient_ID,lip_P2_Sph_Sph(d16:1),lip_P2_Sph_Sph(d18:1),lip_P2_Sph_Sph(d18:2),lip_P1_Cer_Cer(d16:1/16:0),lip_P2_Cer_Cer(d16:1/16:0),lip_P1_Cer_Cer(d16:1/18:0),lip_P2_Cer_Cer(d16:1/18:0),lip_P1_Cer_Cer(d16:1/20:0),lip_P2_Cer_Cer(d16:1/20:0),...,gene_P1_BRCA1_SNP,gene_P1_BRCA2_SNP,gene_P1_CHEK2_SNP,ADT_start_fail,ADT_fail_relaspe,ADT_start_death,1= death; 0=alive,gap_death_consent,gap_surv_time_class,ADT_if_fail
0,1028,0.213523,0.103455,0.163318,0.212065,0.212065,0.166905,0.166905,0.082406,0.082406,...,0,0,0,1734.0,54.0,3208.0,1.0,3219,1,2
1,1066,0.203464,0.082463,0.126036,0.303934,0.303934,0.226785,0.226785,0.117198,0.117198,...,0,0,0,NaN,NaN,NaN,0.0,2464,1,0
2,1040,0.255040,0.148154,0.187509,0.264274,0.264274,0.238348,0.238348,0.184153,0.184153,...,0,1,0,NaN,NaN,2323.0,0.0,2323,1,0
3,1058,0.253185,0.157883,0.213696,0.245078,0.245078,0.197306,0.197306,0.094201,0.094201,...,0,0,1,318.0,0.0,581.0,1.0,581,0,2
4,1059,0.246208,0.099137,0.183114,0.186032,0.186032,0.129798,0.129798,0.090999,0.090999,...,0,0,0,NaN,NaN,2608.0,0.0,2608,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,1530,0.340023,0.170482,0.296729,0.291846,0.291846,0.296924,0.296924,0.258115,0.258115,...,0,0,0,NaN,NaN,34.0,0.0,1,0,0
67,1531,0.349625,0.152349,0.257428,0.259709,0.259709,0.124592,0.124592,0.125189,0.125189,...,0,0,0,NaN,NaN,141.0,0.0,174,0,0
68,1522,0.447353,0.122583,0.262393,0.477925,0.477925,0.346937,0.346937,0.291926,0.291926,...,0,0,0,NaN,NaN,3750.0,1.0,706,0,0
69,1514,0.536190,0.173840,0.155828,0.203830,0.203830,0.121254,0.121254,0.061686,0.061686,...,0,0,0,2355.0,NaN,3453.0,1.0,1098,0,2


In [15]:
# full-feature-data : point2_gene + lip_specie_binary

merge_key = 'Patient_ID'
new1 = pd.merge(lip_table_specie_binary,gene_table2,on = merge_key)
# new2 = pd.merge(new1, clinc_table1_new,on = merge_key)
p2_spicie_binary_single = pd.merge(new1, clinc_target_2,on = merge_key)

print('overlapping patients:',-len(set(p2_spicie_binary_single['Patient_ID']))+len((p2_spicie_binary_single['Patient_ID'])))



# new3.to_csv('../data/processed/all_feature_p1_lip_specie_single.csv',index=False)
p2_spicie_binary_single.to_csv('../data/processed/all_feature_p2_spicie_binary_single.csv',index=False)

p2_spicie_binary_single



overlapping patients: 0


,Patient_ID,lip_P2_Sph(d16:1)_Sph_high_25,lip_P2_Sph(d16:1)_Sph_low_25,lip_P2_Sph(d18:1)_Sph_high_25,lip_P2_Sph(d18:1)_Sph_low_25,lip_P2_Sph(d18:2)_Sph_high_25,lip_P2_Sph(d18:2)_Sph_low_25,lip_P1_Cer_Cer(d16:1/16:0)_high,lip_P1_Cer_Cer(d16:1/16:0)_low,lip_P2_Cer(d16:1/16:0)_Cer_high_5,...,lip_P1_TG(O) [NL]_TG(O-54:4) [NL-17:1]_low,lip_P1_TG(O) [NL]_TG(O-54:4) [NL-18:2]_high,lip_P1_TG(O) [NL]_TG(O-54:4) [NL-18:2]_low,lip_signature,gene_P2_AR_Amp,gene_P2_RB1_Del,gene_P2_TP53_SNP,gap_death_consent,1= death; 0=alive,gap_surv_time_class
0,1001,1,0,0,1,0,1,1,0,1,...,0,1,0,0,0,0,0,2062,1.0,1
1,1010,1,0,1,0,1,0,0,1,0,...,1,0,1,0,1,0,1,655,1.0,2
2,1019,1,0,1,0,0,1,0,1,0,...,0,1,0,1,0,0,0,542,1.0,0
3,1046,0,1,0,1,0,1,1,0,1,...,1,1,0,0,0,1,1,477,1.0,0
4,1011,0,1,0,1,1,0,1,0,1,...,1,0,1,1,0,0,1,1418,1.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,1495,1,0,1,0,0,1,1,0,1,...,1,0,1,1,0,0,0,1289,1.0,2
140,1496,1,0,1,0,0,1,1,0,1,...,1,0,1,0,0,0,1,1249,1.0,2
141,1505,1,0,0,1,1,0,0,1,0,...,0,0,1,1,0,0,0,2150,0.0,1
142,1506,0,1,0,1,0,1,0,1,0,...,0,0,1,1,0,1,1,99,1.0,0


In [18]:
# full-feature-data : point2_gene + lip_specie_binary_standard

merge_key = 'Patient_ID'
new1 = pd.merge(lip_table_specie_binary_standard,gene_table2,on = merge_key)
# new2 = pd.merge(new1, clinc_table1_new,on = merge_key)
p2_spicie_binary_single_standard = pd.merge(new1, clinc_target_2,on = merge_key)

print('overlapping patients:',-len(set(p2_spicie_binary_single_standard['Patient_ID']))+len((p2_spicie_binary_single_standard['Patient_ID'])))



# new3.to_csv('../data/processed/all_feature_p1_lip_specie_single.csv',index=False)
p2_spicie_binary_single_standard.to_csv('../data/processed/all_feature_p2_spicie_binary_standard_single.csv',index=False)

p2_spicie_binary_single_standard



overlapping patients: 0


,Patient_ID,lip_P2_Sph_Sph(d16:1),lip_P2_Sph_Sph(d18:1),lip_P2_Sph_Sph(d18:2),lip_P1_Cer_Cer(d16:1/16:0),lip_P2_Cer_Cer(d16:1/16:0),lip_P1_Cer_Cer(d16:1/18:0),lip_P2_Cer_Cer(d16:1/18:0),lip_P1_Cer_Cer(d16:1/20:0),lip_P2_Cer_Cer(d16:1/20:0),...,lip_P1_TG(O) [NL]_TG(O-54:3) [NL-18:1],lip_P1_TG(O) [NL]_TG(O-54:4) [NL-17:1],lip_P1_TG(O) [NL]_TG(O-54:4) [NL-18:2],lip_signature,gene_P2_AR_Amp,gene_P2_RB1_Del,gene_P2_TP53_SNP,gap_death_consent,1= death; 0=alive,gap_surv_time_class
0,1001,1,0,0,1,1,1,1,1,1,...,1,1,1,1,0,0,0,2062,1.0,1
1,1010,1,1,1,0,0,0,0,0,0,...,1,0,0,1,1,0,1,655,1.0,2
2,1019,1,1,0,0,0,0,0,0,0,...,1,1,1,0,0,0,0,542,1.0,0
3,1046,0,0,0,1,1,1,1,1,1,...,1,0,1,1,0,1,1,477,1.0,0
4,1011,0,0,1,1,1,1,1,1,1,...,0,0,0,0,0,0,1,1418,1.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,1495,1,1,0,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1289,1.0,2
140,1496,1,1,0,1,1,0,0,0,0,...,0,0,0,1,0,0,1,1249,1.0,2
141,1505,1,0,1,0,0,1,1,0,0,...,0,1,0,0,0,0,0,2150,0.0,1
142,1506,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,1,1,99,1.0,0


In [16]:
# full-feature-data : point2_gene + lip_specie_continues

merge_key = 'Patient_ID'
new1 = pd.merge(lip_table_specie_continues,gene_table2,on = merge_key)
# new2 = pd.merge(new1, clinc_table1_new,on = merge_key)
p2_spicie_cont_single = pd.merge(new1, clinc_target_2,on = merge_key)

print('overlapping patients:',-len(set(p2_spicie_cont_single['Patient_ID']))+len((p2_spicie_cont_single['Patient_ID'])))



# new3.to_csv('../data/processed/all_feature_p1_lip_specie_single.csv',index=False)
p2_spicie_cont_single.to_csv('../data/processed/all_feature_p2_spicie_cont_single.csv',index=False)

p2_spicie_cont_single



overlapping patients: 0


,Patient_ID,lip_P2_Sph_Sph(d16:1),lip_P2_Sph_Sph(d18:1),lip_P2_Sph_Sph(d18:2),lip_P1_Cer_Cer(d16:1/16:0),lip_P2_Cer_Cer(d16:1/16:0),lip_P1_Cer_Cer(d16:1/18:0),lip_P2_Cer_Cer(d16:1/18:0),lip_P1_Cer_Cer(d16:1/20:0),lip_P2_Cer_Cer(d16:1/20:0),...,lip_P1_TG(O) [NL]_TG(O-54:3) [NL-18:1],lip_P1_TG(O) [NL]_TG(O-54:4) [NL-17:1],lip_P1_TG(O) [NL]_TG(O-54:4) [NL-18:2],lip_signature,gene_P2_AR_Amp,gene_P2_RB1_Del,gene_P2_TP53_SNP,gap_death_consent,1= death; 0=alive,gap_surv_time_class
0,1001,0.495112,0.054354,0.216575,0.294450,0.294450,0.305661,0.305661,0.181769,0.181769,...,0.091286,0.057364,0.170173,0.291482,0,0,0,2062,1.0,1
1,1010,0.400186,0.137942,0.276835,0.240365,0.240365,0.223917,0.223917,0.171905,0.171905,...,0.079611,0.020748,0.109859,0.261832,1,0,1,655,1.0,2
2,1019,0.368722,0.155415,0.169811,0.174938,0.174938,0.135445,0.135445,0.144821,0.144821,...,0.086322,0.109468,0.363641,0.186702,0,0,0,542,1.0,0
3,1046,0.269143,0.078977,0.222303,0.327287,0.327287,0.319015,0.319015,0.226542,0.226542,...,0.086288,0.040277,0.261594,0.340631,0,1,1,477,1.0,0
4,1011,0.216670,0.047586,0.258964,0.441193,0.441193,0.345246,0.345246,0.271135,0.271135,...,0.032534,0.005912,0.047759,0.162088,0,0,1,1418,1.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,1495,0.419702,0.159182,0.187797,0.408117,0.408117,0.410168,0.410168,0.285887,0.285887,...,0.029835,0.018771,0.086957,0.206957,0,0,0,1289,1.0,2
140,1496,0.420144,0.184309,0.177121,0.316179,0.316179,0.166595,0.166595,0.104448,0.104448,...,0.048295,0.017926,0.123085,0.259028,0,0,1,1249,1.0,2
141,1505,0.433431,0.113845,0.309811,0.225431,0.225431,0.252451,0.252451,0.142770,0.142770,...,0.058747,0.043795,0.148138,0.237137,0,0,0,2150,0.0,1
142,1506,0.271263,0.091081,0.179829,0.176416,0.176416,0.090956,0.090956,0.125632,0.125632,...,0.065104,0.046190,0.126076,0.171522,0,1,1,99,1.0,0


In [25]:
# analysis the t-test and p-value
from scipy.stats import ttest_ind
v1 = np.random.normal(size=10)+5
v2 = np.random.normal(size=100)

res = ttest_ind(v1, v2).pvalue
res

1.5849797810371318e-26

In [29]:
Cer_lip_feature_name_P1 = 
Cer_lip_feature_P2 = 
list(p2_spicie_cont_single.columns)

['Patient_ID',
 'lip_P2_Sph_Sph(d16:1)',
 'lip_P2_Sph_Sph(d18:1)',
 'lip_P2_Sph_Sph(d18:2)',
 'lip_P1_Cer_Cer(d16:1/16:0)',
 'lip_P2_Cer_Cer(d16:1/16:0)',
 'lip_P1_Cer_Cer(d16:1/18:0)',
 'lip_P2_Cer_Cer(d16:1/18:0)',
 'lip_P1_Cer_Cer(d16:1/20:0)',
 'lip_P2_Cer_Cer(d16:1/20:0)',
 'lip_P1_Cer_Cer(d16:1/22:0)',
 'lip_P2_Cer_Cer(d16:1/22:0)',
 'lip_P1_Cer_Cer(d16:1/23:0)',
 'lip_P2_Cer_Cer(d16:1/23:0)',
 'lip_P1_Cer_Cer(d16:1/24:0)',
 'lip_P2_Cer_Cer(d16:1/24:0)',
 'lip_P1_Cer_Cer(d16:1/24:1)',
 'lip_P2_Cer_Cer(d16:1/24:1)',
 'lip_P1_Cer_Cer(d17:1/16:0)',
 'lip_P2_Cer_Cer(d17:1/16:0)',
 'lip_P1_Cer_Cer(d17:1/18:0)',
 'lip_P2_Cer_Cer(d17:1/18:0)',
 'lip_P1_Cer_Cer(d17:1/20:0)',
 'lip_P2_Cer_Cer(d17:1/20:0)',
 'lip_P1_Cer_Cer(d17:1/22:0)',
 'lip_P2_Cer_Cer(d17:1/22:0)',
 'lip_P1_Cer_Cer(d17:1/23:0)',
 'lip_P2_Cer_Cer(d17:1/23:0)',
 'lip_P1_Cer_Cer(d17:1/24:0)',
 'lip_P2_Cer_Cer(d17:1/24:0)',
 'lip_P1_Cer_Cer(d17:1/24:1)',
 'lip_P2_Cer_Cer(d17:1/24:1)',
 'lip_P1_Cer_Cer(d18:1/14:0)',
 'lip_P2